In [1]:
import datetime
import glob
import os

import numpy as np
import pandas as pd

import rasterio
import cartopy
import shapely
import geopandas as gpd

import matplotlib.pyplot as plt

# Plot some risk and hazard maps

# 1. Constants for plotting

In [2]:
#raster resolution in meters
pixel_size=10000

#bounds of output grid in lat/lon
output_min_lon = -168
output_max_lon = -141
output_min_lat = 69
output_max_lat = 73

# Lat/Lon CRS
in_crs = {'init': 'epsg:4326'}

# Equal area projection
out_crs={'init': 'epsg:3338'}

In [3]:
# convert the bounding box to our output projection
bounding_box = shapely.geometry.box(output_min_lon, output_min_lat, output_max_lon, output_max_lat)
bbDF = gpd.GeoDataFrame(crs=in_crs,geometry=[bounding_box])
bbDF = bbDF.to_crs(out_crs)

In [4]:
x_min, y_min, x_max, y_max = bbDF.iloc[0].geometry.bounds
x_res = int((x_max - x_min) / pixel_size)
y_res = int((y_max - y_min) / pixel_size)
print('Output dimensions: ' + str(x_res) + ' x ' + str(y_res))

Output dimensions: 108 x 43


In [5]:
# map features
crs = cartopy.crs.epsg(3338)

land_10m = cartopy.feature.NaturalEarthFeature('physical', 'land', '10m',
                                        edgecolor='gray',
                                        facecolor=cartopy.feature.COLORS['land'])

# 2. Calculate risk

Following Sepp-Neves, risk $R$ is calculated as

$R_s = H_s * I_s$

where $H$ is the oil spill hazard and $I$ is the vulnerability.  Here we treat each species as a seperate vulernability.  In Sepp-Neves $H_s$ was integrated across each region $s$ then multiplied by the scalar $I_s$.  Here we have more ganular data, so instead we integrate $R_s$ across each region $s$.

In [6]:
out_dir = '/data/assets/osra-2017/analysis_outputs/'

In [7]:
ship_types = ['anti-pollution_vessel',
              'cargo',
              'fishing',
              'icebreaker',
              'landing_craft',
              'offshore_supply_vessel',
              'passenger',
              'research-survey_vessel',
              'tanker',
              'tug'
             ]

communities = ['Kaktovik',
                'Nuiqsut',
                'Utqiagvik'
                ]

ship_months = {'anti-pollution_vessel': np.arange(7, 12),
                   'cargo': np.arange(8, 12),
                   'fishing': np.arange(8, 12),
                   'icebreaker': np.arange(6, 12),
                   'landing_craft': np.arange(8, 12),
                   'offshore_supply_vessel': np.arange(8, 12),
                   'passenger': np.arange(6, 12),
                   'research-survey_vessel': np.arange(7, 12),
                   'tanker': np.arange(8, 12),
                   'tug': np.arange(7, 12)
                  }

month_map = {i: datetime.datetime(2017, i, 1).strftime('%B') for i in range(1, 13)}

In [8]:
risk_score = pd.DataFrame({'community': [],
                           'ship_type': [],
                           'month': [],
                           'species': [],
                           'risk_score': []})

In [9]:
calc = False
if calc:
    for ship_type in ship_types:
        for community in communities:
            for month in ship_months[ship_type]:
                fname1 = os.path.join(out_dir, 'monthly_is_{}-{}-*-{}.tiff'.format(ship_type,
                                                                                   community,
                                                                                   month_map[month].lower()))
                fspecies = glob.glob(fname1)
                species = set([os.path.basename(f).split(community+'-')[1].split('-')[0] for f in fspecies])
                for s in species:
                    fname = 'monthly_is_{}-{}-{}-{}.tiff'.format(ship_type,
                                                                 community,
                                                                 s,
                                                                 month_map[month].lower()
                                                                )
                    fpath = os.path.join(out_dir, fname)
                    with rasterio.open(fpath) as ds:
                        data = ds.read()
                        rs = data.sum()
                        risk_score = risk_score.append({'community': community,
                                                        'ship_type': ship_type,
                                                        'month': month,
                                                        'species': s,
                                                        'risk_score': data.sum()}, ignore_index=True)
                        
    risk_score.to_csv('../../outputs/OSRA/risk_score.csv')
else:
    risk_score = pd.read_csv('../../outputs/OSRA/risk_score.csv')
    risk_score = risk_score.loc[:, ~risk_score.columns.str.contains('^Unnamed')]

In [10]:
risk_score.head()

,community,month,risk_score,ship_type,species
0,Kaktovik,7.0,0.000000,anti-pollution_vessel,wolfandwolverine
1,Kaktovik,7.0,0.000000,anti-pollution_vessel,broadwhitefish
2,Kaktovik,7.0,0.001824,anti-pollution_vessel,beardedseal
3,Kaktovik,7.0,0.000000,anti-pollution_vessel,walrus
4,Kaktovik,7.0,0.000000,anti-pollution_vessel,bowhead


In [11]:
community_rs = risk_score.groupby('community')

In [12]:
annual_max = community_rs.sum().max().risk_score

## 2.1 Annual risk by community

In [13]:
tif_out_dir = '../../outputs/OSRA/spatial-risk-tifs/community'
in_dir = '../../outputs/subsistence-actual/'

In [14]:
for community in communities:
    fname = os.path.join(out_dir, 'monthly_is_*-{}-*.tiff'.format(community))
    files = glob.glob(fname)
    files.sort()
    
    # set up data size
    with rasterio.open(files[0]) as ds:
        profile = ds.profile
        data = ds.read()
    data = np.zeros_like(data)
    
    # change dtype
    profile.update(dtype=rasterio.float32, count=1)
    
    for f in files:
        with rasterio.open(f) as ds:
            tmp = ds.read()
            tmp = np.nan_to_num(tmp)
            data += tmp
            #print('- {} {} {}'.format(os.path.basename(f), np.nansum(tmp), np.nansum(data)))
    print(np.nansum(data))
    data = data/annual_max
    print(np.nansum(data))

    # save tiff
    out_path = os.path.join(tif_out_dir, '{}-allresources-allmonths.tif'.format(community.lower()))
    with rasterio.open(out_path, 'w', **profile) as out:
        out.write(data[0, :, :].astype(rasterio.float32), 1)

3.733577648609244
0.03026097347759598
27.552673228121716
0.22331682698594452
53.93374796566751
0.43713774570735453


In [15]:
# set up data size
with rasterio.open(files[0]) as ds:
    example_profile = ds.profile
    data = ds.read()
example_data = np.zeros_like(data)
    
# change dtype
example_profile.update(dtype=rasterio.float32, count=1)

## 2.2 Monthly risk by community

### 2.2.1 Make tifs

In [17]:
tif_out_dir = '../../outputs/OSRA/spatial-risk-tifs/community-month'
in_dir = '../../outputs/subsistence-actual/'

In [18]:
for community in communities:
    for month in range(1, 13):
        month_name = month_map[month]
        fname = os.path.join(out_dir, 'monthly_is_*-{}-*-{}.tiff'.format(community, month_name.lower()))
        files = glob.glob(fname)
        files.sort()

        # set up data size
        data = np.zeros_like(example_data)

        for f in files:
            with rasterio.open(f) as ds:
                tmp = ds.read()
                tmp = np.nan_to_num(tmp)
                data += tmp
        print(np.nansum(data))
        data = data/annual_max
        print(np.nansum(data))

        # save tiff
        out_path = os.path.join(tif_out_dir, '{}-allresources-{}.tif'.format(community.lower(), month_name.lower()))
        with rasterio.open(out_path, 'w', **example_profile) as out:
            out.write(data[0, :, :].astype(rasterio.float32), 1)

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.002431414774532708
1.9706829462238902e-05
0.10449220564131281
0.0008469184683235743
3.4365186670462773
0.027853284443538497
0.1728784898243475
0.0014011952844666688
0.01725687132277375
0.0001398684518050042
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.06154574394589456
0.0004988336390698569
0.15437500438640123
0.001251222916196961
4.398670865545386
0.035651612187180784
20.705510106127733
0.16781997085155664
2.1925824332185977
0.017771072441411812
0.03998907489770422
0.0003241149505284272
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.7760698499582795
0.0062901140291263015
12.967355705480973
0.10510155245446237
20.619387958361347
0.16712194330943153
16.761532471143205
0.1358536870773446
2.8094019807237007
0.02277044883698981
0.0
0.0


## 2.3 Annual risk by community, species

Normalized by maximum annual risk of all species

In [19]:
tif_out_dir = '../../outputs/OSRA/spatial-risk-tifs/community-resource'
in_dir = '../../outputs/subsistence-actual/'

In [20]:
for community in communities:
    for species in risk_score['species'].unique():
        fname = os.path.join(out_dir, 'monthly_is_*-{}-{}-*.tiff'.format(community, species))
        files = glob.glob(fname)
        files.sort()

        # set up data size
        data = np.zeros_like(example_data)

        for f in files:
            with rasterio.open(f) as ds:
                tmp = ds.read()
                tmp = np.nan_to_num(tmp)
                data += tmp
        print(np.nansum(data))
        data = data/annual_max
        print(np.nansum(data))

        # save tiff
        out_path = os.path.join(tif_out_dir, '{}-{}-allmonths.tif'.format(community.lower(), species))
        with rasterio.open(out_path, 'w', **example_profile) as out:
            out.write(data[0, :, :].astype(rasterio.float32), 1)

0.0033613291859863103
2.7243867121523666e-05
0.004337504301813714
3.515585183692013e-05
0.2681232297793092
0.002173162234378971
0.12545447717388303
0.0010168195129994664
2.9281023116115485
0.023732525403448005
0.006712615261872853
5.44063339570576e-05
0.0
0.0
0.2652620719419827
0.0021499722997963045
0.10810422269640772
0.0008761941825558915
0.01301251290544114
0.00010546755551076042
0.002432365147371253
1.9714532317239585e-05
0.004337504301813714
3.515585183692013e-05
0.004337504301813714
3.515585183692013e-05
0.008443889586628821
6.843846382139793e-05
0.0
0.0
5.564957733226642
0.0451044692834615
0.01929068777015152
0.00015635271204138176
15.643846480255476
0.1267947443393697
0.0
0.0
0.0
0.0
5.1766724608720525
0.041957383181518454
0.1348007614080584
0.00109257196438609
0.0
0.0
1.0046612150027034
0.008142867041345961
0.0
0.0
0.0
0.0
0.05210308442020468
0.00042230005751402384
0.011113972592393644
9.007972017789202e-05
9.15482157189643
0.07420062975855783
7.980847699251152
0.0646854688145

## 2.4 Risk score by species, month

In [21]:
tif_out_dir = '../../outputs/OSRA/spatial-risk-tifs/community-resource-month'
in_dir = '../../outputs/subsistence-actual/'

In [22]:
for community in communities:
    for species in risk_score['species'].unique():
        for month in range(1, 13):
            month_name = month_map[month]
            fname = os.path.join(out_dir, 'monthly_is_*-{}-{}-{}.tiff'.format(community, species, month_name.lower()))
            files = glob.glob(fname)
            files.sort()

            # set up data size
            data = np.zeros_like(example_data)

            for f in files:
                with rasterio.open(f) as ds:
                    tmp = ds.read()
                    tmp = np.nan_to_num(tmp)
                    data += tmp
            print(np.nansum(data))
            data = data/annual_max
            print(np.nansum(data))

            # save tiff
            out_path = os.path.join(tif_out_dir, '{}-{}-{}.tif'.format(community.lower(), species, month_name.lower()))
            with rasterio.open(out_path, 'w', **example_profile) as out:
                out.write(data[0, :, :].astype(rasterio.float32), 1)

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.000986530380100015
7.995915038266206e-06
0.002374798805886295
1.924795208325746e-05
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.004337504301813714
3.515585183692013e-05
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.001215707387266354
9.853414731119451e-06
0.03477260317126966
0.00028183499081764767
0.23213491922077323
0.0018814738288302035
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.009145486658597164
7.412496946959344e-05
0.0869046842463434
0.0007043700687561856
0.02940430626894245
0.00023832447477368737
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.028662670477502847
0.00023231345180135154
2.777669125940841
0.022513251272795527
0.12177051519320473
0.000986960678851125
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.004739554501672823
3.841450388052519e-05
0